In [15]:
import pandas as pd
import efinance as ef

stock_concepts = pd.read_csv("stock_concept.csv")
concept_dic = {}
code2name = {}
for _, row in stock_concepts.iterrows():
    t, n, c = row['板块类型'], row['板块名称'], row['板块代码']
    if t not in concept_dic:
        concept_dic[t] = {}
    if n not in concept_dic[t]:
        concept_dic[t][n] = c
    code2name[c] = n

In [16]:
code2name

{'BK1092': '麒麟电池',
 'BK1000': '宁组合',
 'BK0999': '茅指数',
 'BK0988': '钠离子电池',
 'BK0968': '固态电池',
 'BK1093': '汽车一体化压铸',
 'BK1119': 'PLC概念',
 'BK1097': 'TOPCon电池',
 'BK1059': '百元股',
 'BK0743': '深证100R',
 'BK0899': 'CRO',
 'BK0984': '华为汽车',
 'BK1099': '轮毂电机',
 'BK1082': '噪声防治',
 'BK0898': '胎压监测',
 'BK0951': '刀片电池',
 'BK0890': 'MLCC',
 'BK1113': '复合集流体',
 'BK1014': '发电机概念',
 'BK0811': '超级品牌',
 'BK1089': '汽车热管理',
 'BK0985': '换电概念',
 'BK1073': '啤酒概念',
 'BK1108': '科创板做市股',
 'BK0865': '电子烟',
 'BK0960': '无线充电',
 'BK0872': '青蒿素',
 'BK1052': '动力电池回收',
 'BK1094': '钙钛矿电池',
 'BK0896': '白酒',
 'BK1006': '碳基材料',
 'BK0644': '特斯拉',
 'BK0987': '盐湖提锂',
 'BK1121': '第四代半导体',
 'BK0536': '基金重仓',
 'BK0638': '创业成份',
 'BK0574': '锂电池',
 'BK0989': '储能',
 'BK0893': '无线耳机',
 'BK0986': 'CAR-T细胞疗法',
 'BK1107': '科创板做市商',
 'BK0842': '富士康',
 'BK1063': '重组蛋白',
 'BK0900': '新能源车',
 'BK1085': '痘病毒防治',
 'BK0875': 'ETC',
 'BK0823': '养老金',
 'BK0500': 'HS300_',
 'BK0700': '充电桩',
 'BK1002': '激光雷达',
 'BK1065': '气溶胶检测',
 'BK0711': '券商概

In [3]:
concept_histories = {}
for concept_type in concept_dic:
    codes = list(concept_dic[concept_type].values())
    concept_histories[concept_type] = ef.stock.get_quote_history(codes)


Processing => BK0482: 100%|██████████| 86/86 [00:07<00:00, 10.87it/s]


In [11]:
import datetime
curr_time = datetime.datetime.now()
curr_time + datetime.timedelta(days=-1)
curr_time.strftime("%Y-%m-%d")
time_ranges = []
time_unit = 7
for i in range(120):
    start_time = curr_time + datetime.timedelta(days=-i - 7)
    end_time = curr_time + datetime.timedelta(days=-i)
    time_ranges.append((start_time.strftime("%Y-%m-%d"), end_time.strftime("%Y-%m-%d")))

In [41]:
concept_histories['概念板块']['BK1119'].query("(日期 > '2022-12-08') & (日期 < '2022-12-18')")

971.23

In [18]:
time_ranges[0]

('2022-12-30', '2023-01-06')

In [42]:
def get_change(df, start_time, end_time):
    cut_data = df.query("(日期 > '%s') & (日期 < '%s')"%(start_time, end_time))
    if cut_data.size == 0:
        return 0
    open_price = cut_data.iloc[0]['开盘']
    close_price = cut_data.iloc[-1]['收盘']
    change = (close_price - open_price) / open_price
    return change

get_change(concept_histories['概念板块']['BK1119'], '2022-12-08', '2022-12-18')


-0.026169873562411662

In [43]:
start_time_concepts={}
for start_time, end_time in time_ranges:
    start_time_concepts[start_time] = []
    for t in concept_histories:
        for code in concept_histories[t]:
            change = get_change(concept_histories[t][code], start_time, end_time)
            start_time_concepts[start_time].append((code2name[code], change))
    start_time_concepts[start_time].sort(key=lambda x: x[1], reverse=True)

In [45]:
days = list(start_time_concepts.keys())
days.sort()
for d in days:
    print(d, [(_[0], '%.1f%%'%(_[1] * 100)) for _ in start_time_concepts[d][:10] if '昨日' not in _[0]])
    reverse_list =  start_time_concepts[d][-10:]
    reverse_list.sort(key=lambda x:x[1], reverse=False)
    print(d, [(_[0], '%.1f%%'%(_[1] * 100)) for _ in reverse_list if '昨日' not in _[0]])
    print("----------------------------------")

2022-09-02 [('超超临界发电', '8.3%'), ('熔盐储能', '8.0%'), ('光伏设备', '7.9%'), ('TOPCon电池', '7.3%'), ('抽水蓄能', '7.1%'), ('电力行业', '6.9%'), ('环氧丙烷', '6.7%'), ('统一大市场', '6.7%'), ('超导概念', '6.6%')]
2022-09-02 [('鸡肉概念', '-4.8%'), ('游戏', '-4.7%'), ('猪肉概念', '-4.2%'), ('宠物经济', '-3.5%'), ('数字阅读', '-3.5%'), ('虚拟数字人', '-3.4%'), ('云游戏', '-3.4%'), ('农牧饲渔', '-3.4%'), ('快手概念', '-3.4%'), ('华为昇腾', '-3.3%')]
----------------------------------
2022-09-03 [('租售同权', '8.1%'), ('基本金属', '8.0%'), ('超超临界发电', '6.8%'), ('环氧丙烷', '6.5%'), ('抽水蓄能', '6.5%'), ('统一大市场', '6.4%'), ('房地产开发', '6.3%'), ('电力行业', '6.2%'), ('稀缺资源', '6.0%'), ('物流行业', '5.9%')]
2022-09-03 [('鸡肉概念', '-5.4%'), ('游戏', '-4.7%'), ('猪肉概念', '-4.5%'), ('宠物经济', '-4.0%'), ('天基互联', '-4.0%'), ('云游戏', '-3.9%'), ('农牧饲渔', '-3.6%'), ('快手概念', '-3.6%'), ('数字阅读', '-3.5%'), ('虚拟数字人', '-3.5%')]
----------------------------------
2022-09-04 [('租售同权', '8.1%'), ('基本金属', '8.0%'), ('超超临界发电', '6.8%'), ('环氧丙烷', '6.5%'), ('抽水蓄能', '6.5%'), ('统一大市场', '6.4%'), ('房地产开发', '6.3%'), ('电力行业', '6